In [177]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [178]:
df_input=pd.read_csv('./input_fluxo_caixa_SQS310C309.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [179]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
entrada_por_valor=df_input.loc['entrada_por_valor','valor_variavel']
financiamento_0_ou_1=df_input.loc['financiamento_0_ou_1','valor_variavel']
area=df_input.loc['area','valor_variavel']

# Custos Iniciais

In [180]:
itbi_parcelado_0_ou_1=df_input.loc['itbi_parcelado_0_ou_1','valor_variavel']
custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']


# Cálculo financiamento

In [181]:
# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento


In [182]:
if financiamento_0_ou_1==1:
    n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                         # 35 anos
    df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))
    juros_nominal=df_input.loc['juros_nominal','valor_variavel']                        # a.a    
    juros_tr=df_input.loc['tr','valor_variavel']                        # a.a
    juros_nominal=juros_nominal+juros_tr
    custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']
    juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

    # he usual way to test for a NaN is to see if it's equal to itself:
    if entrada_por_valor!=entrada_por_valor:
        # Não digitou o valor da entrada
        if entrada_pp!=entrada_pp:
            print('Digite o valor da entrada ou seu percentual.')
        else:
            valor_entrada=valor_imovel*entrada_pp
    else:
        valor_entrada=entrada_por_valor
    
    valor_finaciado=valor_imovel-valor_entrada
    encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']
    amortizacao=valor_finaciado/n_parcelas
    saldo_devedor=valor_finaciado

else:
    amortizacao=0
    saldo_devedor=0
    n_parcelas=0   
    juros_nominal=0
    custo_avaliacao=0
    juros_efetivo=0  
    valor_finaciado=0
    encargos_finaciamento=0
    valor_entrada=valor_imovel
    df=pd.DataFrame()

In [183]:
total_custos_iniciais=valor_entrada+custo_escritura+custo_registro+custo_avaliacao

In [184]:
for i in range(0,max(24,n_parcelas+1)):             # Se não for financiado seta o df para 24 linhas inicialmente
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

In [185]:
# ITBI
if itbi_parcelado_0_ou_1==1:
    for i in range(0,len(df)):
        if i <10:
            df.loc[i,'itbi']=custo_itbi/10
        else:
            df.loc[i,'itbi']=0
else:
    df.loc[0,'itbi']=custo_itbi
    for i in range(1,len(df)):
        df.loc[0,'itbi']=0

# Custos fixos

In [186]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [187]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [188]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [189]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [190]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir:
# Da venda, da quitação do ITBI e da
for i in range(0,len(df)):
    if i>prazo_venda:
        if df.loc[i,'itbi']!=0:                                     # caso tenha parcela do ITBI ainda para pagar
            df.loc[i,'total']=df.loc[i,'itbi']
        else:
            df.loc[i,'total']=0
        
        # Quitação do saldo devedor e retirada dos custos fixos e parcelas
        df.loc[i,'saldo_devedor']=0 
        df.loc[i,'parcela']=0    
        df.loc[i,'custos_fixos']=0    

    elif i==prazo_venda:                                            
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']
        
    

Comissão: 12450.0


In [191]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

# Filtra apenas até a célula de venda
df=df[df['total']!=0]
df

,saldo_devedor,parcela,itbi,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0.00,0.00,"2,040.00",881.67,"682,800.00",0.00,0.00,"685,721.67",2022-10-15
1,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2022-11-15
2,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2022-12-15
3,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2023-01-15
4,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2023-02-15
5,0.00,0.00,"2,040.00",881.67,0.00,0.00,0.00,"2,921.67",2023-03-15
6,0.00,0.00,"2,040.00",881.67,0.00,0.00,0.00,"2,921.67",2023-04-15
7,0.00,0.00,"2,040.00",881.67,0.00,0.00,0.00,"2,921.67",2023-05-15
8,0.00,0.00,"2,040.00",881.67,0.00,0.00,"-817,550.00","-814,628.33",2023-06-15
9,0.00,0.00,"2,040.00",0.00,0.00,0.00,0.00,"2,040.00",2023-07-15


In [192]:
# Importa output da BcB_forcast (os dois diretórios no mesmo caminho)
df_juros_real=pd.read_csv(r'../BcB_Forcast/output/num_indice_selic_ipca_diferenca.csv', index_col=0)
df_juros_real

,num_indice_selic,num_indice_ipca,dif_num_indice
2022-09-29,100.00,100.00,100.00
2022-09-30,100.04,99.99,100.04
2022-10-01,100.07,100.01,100.06
2022-10-02,100.11,100.02,100.09
2022-10-03,100.14,100.03,100.11
...,...,...,...
2024-09-26,125.28,109.38,115.90
2024-09-27,125.31,109.38,115.92
2024-09-28,125.33,109.39,115.94
2024-09-29,125.36,109.40,115.96


In [193]:
def busca_num_indice_selic(data):
    #converte data em string
    data=datetime.strftime(data,'%Y-%m-%d')    
    return df_juros_real.loc[data,'num_indice_selic']
df['num_selic']=df.apply(lambda x: busca_num_indice_selic(x.data),axis=1)

def corrige_selic(total,num_selic):
    # Número índice de referência (data da venda)
    num_indice_ref=df.loc[df.index[-1],'num_selic']
    total=total*num_indice_ref/num_selic
    return total
df['total_selic']=df.apply(lambda x: corrige_selic(x.total,x.num_selic),axis=1)

df

,saldo_devedor,parcela,itbi,custos_fixos,custos_iniciais,reforma,venda,total,data,num_selic,total_selic
0,0.00,0.00,"2,040.00",881.67,"682,800.00",0.00,0.00,"685,721.67",2022-10-15,100.57,"754,927.94"
1,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2022-11-15,101.67,"16,794.40"
2,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2022-12-15,102.76,"16,616.26"
3,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2023-01-15,103.89,"16,435.53"
4,0.00,0.00,"2,040.00",881.67,0.00,"12,500.00",0.00,"15,421.67",2023-02-15,105.03,"16,257.14"
5,0.00,0.00,"2,040.00",881.67,0.00,0.00,0.00,"2,921.67",2023-03-15,106.07,"3,049.75"
6,0.00,0.00,"2,040.00",881.67,0.00,0.00,0.00,"2,921.67",2023-04-15,107.24,"3,016.48"
7,0.00,0.00,"2,040.00",881.67,0.00,0.00,0.00,"2,921.67",2023-05-15,108.38,"2,984.75"
8,0.00,0.00,"2,040.00",881.67,0.00,0.00,"-817,550.00","-814,628.33",2023-06-15,109.57,"-823,178.32"
9,0.00,0.00,"2,040.00",0.00,0.00,0.00,0.00,"2,040.00",2023-07-15,110.72,"2,040.00"


In [194]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Lucro: ',-x['total'].sum())

    amounts_selic=list(x['total_selic'])
    t_aa=xirr(dates, amounts_selic)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a. acima da selic (%): ',t_aa*100)
    print('TII a.m. acima da selic (%): ',t_am*100)
    print('Lucro: ',-x['total_selic'].sum())
    
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [195]:
cal_irr(df)

TII a.a.(%):  11.840558435665109
TII a.m.(%):  0.9368956932371164
Lucro:  56415.00000000012
TII a.a. acima da selic (%):  -1.6695954974207177
TII a.m. acima da selic (%):  -0.14020914400149076
Lucro:  -8943.914667625097


In [196]:
valor_m2_aquisicao=valor_imovel/area
print(f'm2 aquisição: {valor_m2_aquisicao}')
valor_m2_venda=valor_venda/area
print(f'm2 reforma: {obra/area}')
print(f'm2 venda: {valor_m2_venda}')


m2 aquisição: 10000.0
m2 reforma: 735.2941176470588
m2 venda: 12205.882352941177


In [197]:
11*66+70

796

In [198]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda